Download dataset

In [1]:
!wget "https://www.etherchain.org/charts/priceUSD/data" -O eth.json

--2018-09-01 12:39:19--  https://www.etherchain.org/charts/priceUSD/data
Resolving www.etherchain.org (www.etherchain.org)... 172.64.129.2, 172.64.128.2
Connecting to www.etherchain.org (www.etherchain.org)|172.64.129.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘eth.json’

eth.json                [      <=>           ] 794.06K   597KB/s    in 1.3s    

2018-09-01 12:39:21 (597 KB/s) - ‘eth.json’ saved [813121]



Import Python dependencies

In [2]:
import json
import datetime

Parse the downloaded data

In [12]:
# DP stores data points from the downloaded dataset
class DP:
    def __init__(self, ts, price):
        self.date = datetime.datetime.utcfromtimestamp(ts)
        self.price = float(price)
        self.diff_pct = None # calculated later
        
    def __repr__(self):
        return f"DP{{date={self.date},price={self.price},diff_pct={self.diff_pct}}}"

# parse JSON
json_data = []
with open('eth.json') as f:
    json_data = json.load(f)

# convert json objects to DP array
data = list(map(lambda r: DP(r["time"], r["value"]), json_data))

# remove DPs with bad price
data = list(filter(lambda dp: dp.price > 0, data))

# only keep DPs from specific date
data = list(filter(lambda dp: dp.date.year == 2018 and dp.date.month == 8, data))

# visualize DPs below
data

[DP{date=2018-08-01 00:04:18,price=432.39,diff_pct=None},
 DP{date=2018-08-01 00:07:22,price=432.88,diff_pct=None},
 DP{date=2018-08-01 00:28:54,price=433.59,diff_pct=None},
 DP{date=2018-08-01 01:04:18,price=431.29,diff_pct=None},
 DP{date=2018-08-01 01:28:55,price=432.0,diff_pct=None},
 DP{date=2018-08-01 01:52:58,price=420.67,diff_pct=None},
 DP{date=2018-08-01 02:04:19,price=417.4,diff_pct=None},
 DP{date=2018-08-01 02:10:08,price=412.43,diff_pct=None},
 DP{date=2018-08-01 02:33:26,price=415.44,diff_pct=None},
 DP{date=2018-08-01 03:04:19,price=416.55,diff_pct=None},
 DP{date=2018-08-01 03:33:27,price=417.85,diff_pct=None},
 DP{date=2018-08-01 04:04:21,price=417.12,diff_pct=None},
 DP{date=2018-08-01 04:33:27,price=418.32,diff_pct=None},
 DP{date=2018-08-01 05:04:22,price=416.96,diff_pct=None},
 DP{date=2018-08-01 05:11:28,price=417.3,diff_pct=None},
 DP{date=2018-08-01 05:29:32,price=416.94,diff_pct=None},
 DP{date=2018-08-01 05:48:48,price=416.89,diff_pct=None},
 DP{date=2018-08-

Create data structure to store transactions

In [13]:
class Tx:
    def __init__(self, buyOrSell, amount, price, date, closed):
        self.typ = buyOrSell
        self.amount = amount # negative for sells
        self.price = price
        self.date = date
        self.closed = closed # for "buy" positions, whether a corresponding "sell" has been traded
    
    def __repr__(self):
        return f"Tx{{typ={self.typ},amount={self.amount},price={self.price},date={self.date},closed={self.closed}}}"

Calculate change %

In [14]:
# lookback is the period for which the change percentage is calculated
LOOKBACK_MIN_SEC = 7 * 24 * 60 * 60
LOOKBACK_MAX_SEC = LOOKBACK_MIN_SEC + 8 * 60 * 60

# clone array
DPs = data.copy()

# traverse DP array and calculate change pct
for i in range(0, len(DPs)):    
    dp = DPs[i]
    prev_dp = None
    
    for j in range(i - 1, 0, -1):
        if LOOKBACK_MIN_SEC <= (dp.date - DPs[j].date).total_seconds() <= LOOKBACK_MAX_SEC:
            prev_dp = DPs[j]
            break
    
    if prev_dp is not None:
        dp.diff_pct = round((dp.price - prev_dp.price) / prev_dp.price * 10000) / 100
    else:
        dp.diff_pct = None

# filter out DPs with invalid diff_pct
DPs = list(filter(lambda dp: dp.diff_pct is not None, DPs))

# visualize below
DPs

[DP{date=2018-08-08 00:40:38,price=376.75,diff_pct=-13.11},
 DP{date=2018-08-08 00:45:53,price=376.67,diff_pct=-13.13},
 DP{date=2018-08-08 01:00:59,price=377.24,diff_pct=-13.0},
 DP{date=2018-08-08 01:45:53,price=378.53,diff_pct=-12.38},
 DP{date=2018-08-08 02:00:59,price=377.84,diff_pct=-10.18},
 DP{date=2018-08-08 02:30:22,price=376.36,diff_pct=-8.75},
 DP{date=2018-08-08 02:45:54,price=377.15,diff_pct=-9.22},
 DP{date=2018-08-08 02:53:38,price=377.12,diff_pct=-9.22},
 DP{date=2018-08-08 03:01:43,price=377.94,diff_pct=-9.03},
 DP{date=2018-08-08 03:45:55,price=377.03,diff_pct=-9.77},
 DP{date=2018-08-08 04:01:43,price=376.29,diff_pct=-9.95},
 DP{date=2018-08-08 04:06:58,price=374.64,diff_pct=-10.18},
 DP{date=2018-08-08 04:45:55,price=373.38,diff_pct=-10.74},
 DP{date=2018-08-08 05:06:58,price=371.99,diff_pct=-10.79},
 DP{date=2018-08-08 05:25:05,price=369.22,diff_pct=-11.52},
 DP{date=2018-08-08 05:45:56,price=372.94,diff_pct=-10.55},
 DP{date=2018-08-08 06:22:19,price=373.18,diff_

Make transactions

In [15]:
# store transactions
txs = []

# how long to keep "buy" positions for
MIN_SEC_KEEP_POS_OPEN = 7 * 24 * 60 * 60
        
# traverse DP array and make trades
for i in range(0, len(DPs)):
    dp = DPs[i]
    
    if dp.diff_pct > 20:
        txs.append(Tx("buy", +1, dp.price, dp.date, False))

    # close any open positions
    for j in range(0, len(txs)):
        if txs[j].typ is "buy" and txs[j].closed is False and (dp.date - txs[j].date).total_seconds() >= MIN_SEC_KEEP_POS_OPEN:
            txs[j].closed = True
            txs.append(Tx("sell", -1, dp.price, dp.date, True))

# visualize transactions
txs

[]

Results

In [29]:
print(f"cash at the end = ${sum(map(lambda tx: tx.amount * tx.price, txs))}")

cash at the end = $85437.83999999674


THE END